# 1141_社會資料分析 作業三
學生：劉晏成

## 題目

以「2018台北大學社會系三鶯調查」（brilliant_working.sav）檔案回答以下問題：
1. 受訪者的平均教育年數（eduy）是否為12年？
2. 不同性別（a1）受訪者的教育程度（a8）是否有顯著差異？
3. 受訪者的教育程度（a8）和其配偶的教育程度（a10）有無顯著差異？

In [1]:
from load import load_sav
import pandas as pd

df = load_sav("../data/brilliant_working.sav")

In [2]:
variable_value_labels = df.attrs['variable_value_labels']
column_names = df.attrs['column_names']
column_names_to_labels = df.attrs['column_names_to_labels']

In [3]:
target_columns = ['eduy', 'a1', 'a8', 'a10']

## Q1: 受訪者的平均教育年數（eduy）是否為12年？

觀察數值分布

In [4]:
df_1 = df[['eduy']].copy()
df_1.value_counts().sort_index()

eduy
0.0       4
3.0       1
6.0      55
8.0       1
9.0      91
12.0    290
13.0      1
14.0     95
15.0      9
16.0     64
18.0    253
20.0      4
Name: count, dtype: int64

由於教育年數為Ratio scale ，並且數值0 有其意義，故不做recoding 也不排除數值

以樣本做母體平均數檢定

* H0: mean = 12
* H1: mean != 12

並且以$\alpha$ < 0.05 為檢定判準

In [5]:
from scipy import stats

mu_0 = 12

# two-sided for for two-sided test; less for case you want to check if the sample mean is less than population mean
t_statistic, p_value = stats.ttest_1samp(df_1['eduy'], mu_0, alternative='two-sided')

pd.DataFrame({
    'Values': [f'{t_statistic:.2f}', f'{p_value:.15e}']
}, index=['statstic', 'p value'])

,Values
statstic,12.31
p value,3.281672369615777e-32


### Answer

t-test 結果為t: 12.31，並且p value < 0.05
故拒絕H0。母體平均數不為12

## Q2: 不同性別（a1）受訪者的教育程度（a8）是否有顯著差異？

In [6]:
df_2 = df[['a1', 'a8']].copy().rename(columns = { 'a1': 'sex', 'a8': 'education' })

In [7]:
df_2_male = df_2.loc[df_2['sex'] == 1.0, 'education']
df_2_female = df_2.loc[df_2['sex'] == 2.0, 'education']

檢視兩性別教育程度的描述性統計結果

In [8]:
pd.DataFrame({'Male': df_2_male, 'Female': df_2_female}).describe()

,Male,Female
count,418.000000,450.000000
mean,11.698565,11.188889
std,6.207835,6.041974
min,1.000000,1.000000
25%,7.000000,7.000000
50%,10.000000,8.000000
75%,19.000000,19.000000
max,21.000000,21.000000


考察兩資料的性質為獨立樣本，進行Levene's test 以及接續之t-test

In [9]:
from statistic import t_test

t_test.compare_independent_samples(df_2_male, df_2_female) 

,Statistic,p-value
Levene's Test for Equal Variances,2.392620,0.122274
t-test (Equal Variance Assumed),1.225481,0.220727
t-test (Equal Variance Not Assumed),1.224257,0.221191


### Answer
**Levene's Test**
H0: 兩母群體變異數相同
H1: 兩母群體變異數不相同
$\alpha = 0.05$

首先檢視表格中Levene's Test 的P Value > 0.05，接受Levenu's test 的H0（兩母群體變異數相等）。

接下觀察母群體變異數相同的T Test 結果

設定：
男性、女性受訪者教育程度平均數為 $\mu_1$、$\mu_2$  
$H_0: \mu_1 = \mu_2$  
$H_1: \mu_1 \neq \mu_2$  
$\alpha = 0.05$

由此設定進一步檢視假設變異數相等的分析結果，P Value > 0.05，不同性別與教育程度無顯著差異。

## Q3: 受訪者的教育程度（a8）和其配偶的教育程度（a10）有無顯著差異？

In [10]:
df_3 = df[['a8', 'a10']].copy().rename(columns = { 'a8': 'education', 'a10': 'spouse_education' })
df_3

,education,spouse_education
0,18.0,18.0
1,18.0,8.0
2,18.0,NaN
3,8.0,NaN
4,6.0,19.0
...,...,...
863,3.0,3.0
864,8.0,8.0
865,3.0,4.0
866,8.0,NaN


檢視兩組資料的缺漏值

In [11]:
pd.DataFrame({ '# of Missing Values': [df_3['education'].isna().sum(), df_3['spouse_education'].isna().sum()] },
             index = ['Education', 'Spouse Education'])

,# of Missing Values
Education,0
Spouse Education,244


首先考察兩資料為相依資料，並且對於教育程度資料而言，缺漏值並無統計意義，因此排除組合中有缺漏值的樣本

In [12]:
df_3_education = df_3.loc[:, 'education']
df_3_cleaned = df_3.dropna()

df_3_education = df_3_cleaned.loc[:, 'education']
df_3_spouse_education = df_3_cleaned.loc[:, 'spouse_education']

df_3_cleaned

,education,spouse_education
0,18.0,18.0
1,18.0,8.0
4,6.0,19.0
5,19.0,6.0
6,3.0,1.0
...,...,...
862,3.0,4.0
863,3.0,3.0
864,8.0,8.0
865,3.0,4.0


二資料描述性統計結果

In [13]:
df_3_cleaned.describe()

,education,spouse_education
count,624.000000,624.000000
mean,10.504808,10.349359
std,5.970498,6.078914
min,1.000000,1.000000
25%,7.000000,6.000000
50%,8.000000,8.000000
75%,18.000000,18.000000
max,21.000000,21.000000


因此二組資料為相依資料，故為檢驗此二資料是否有顯著差異，執行Paired Samples t-test

In [14]:
from statistic import t_test

t_test.compare_paired_samples(df_3_education, df_3_spouse_education)


,Value
diff_mean,0.155449
diff_std,4.916628
t_stat,0.789791
p_value,0.429950


### Answer

設定：受訪者與其配偶的教育程度平均數分別為$\mu_1$、$\mu_1$  
$H_0: \mu_1 = \mu_2$  
$H_1: \mu_1 \neq \mu_2$  
$\alpha = 0.05$

依據Paired Samples T-Test 結果，p value(0.43) > 0.05 ，接受虛無假設；故受訪者教育程度與其配偶教育程度無顯著差異